In [1765]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1766]:
from csv_data import get_vocab_dict
from csv_data import get_sports
from csv_data import csv_to_list
from csv_data import split_data
from csv_data import extract_data

# # TODO : fjern ord der er kategorisert som "egennavn" i ddo_fullforms_2020-08-26.csv

ordered_dict = get_vocab_dict()
df_sport = get_sports()
nationalities = csv_to_list('nat3.csv')
danske_navne = csv_to_list('danmark_navne.csv')
danske_fornavne = csv_to_list('danske_fornavne.csv')
countries = csv_to_list('countries.csv')
navne = csv_to_list('navne.csv')

def get_results_in_data(train_data, train_labels):
    results = []
    for t in range(len(train_data)):
        if train_labels[t] == 1:
            results.append(train_data[t])
    
    return results


train_data, val_data, train_labels, val_labels = split_data(extract_data(df_sport), 5)

train_data_results = get_results_in_data(train_data, train_labels)

# print("Total data: ", len(train_text))
print("Train data length: ", len(train_data), len(train_labels))
print("Validation data length: ", len(val_data),  len(val_labels))


# TODO : create train, validation and test set here


Train data length:  1958 1958
Validation data length:  103 103


In [1767]:
import re
train_results =  get_results_in_data(train_data, train_labels)
train_results_frequent_words = []
for tt in train_results:
    if not re.search(r'\bvinde|vinder|vundet|vandt|sejr|sejren|sejrer|tabe|tabte|taber|slår|slog|nederlag|succes|triumf|sikrer sig|sikrede sig|point|booker billet|ryger ud|ydmyger|kvalificerer|klar til|medaljer|videre|sikrer|sikret', tt, re.IGNORECASE):
        train_results_frequent_words.append(tt)

print(len(train_results))
print(len(train_results_frequent_words))
# train_results_frequent_words

931
202


['BEIJING 2022 . Olympisk top 5: Norsk legende sætter gyldent OL-punktum . En top-5 over dagens mest bemærkelsesværdige øjeblikke og kommende højdepunkter på OL-programmet',
 'GOLF . Tiger Woods fører US Open . Ingen af de tre danskere klarede cuttet efter anden runde af golfturneringen US Open, som føres af Tiger Woods ',
 'Formel 1 . Magnussen reagerer på skuffelse . Haas kæmper med at finde farten i Formel 1-løbene. Søndag blev Kevin Magnussen nummer 17 i Canada ',
 'SPORT . Hov, hvad skete der?: Sindssygt glade Helena Rosendahl Bach bryder den suverænt længste rekordtørke i dansk svømning . Med den første danske rekord i svømning i halvandet år sikrede Helena Rosendahl Bach Danmarks eneste medalje ved EM ',
 'Curling . Danmark et skridt nærmere OL-kval . Kvindelandsholdet i curling nærmer sig den eftertragtede sjetteplads ved VM, som giver adgang til vinter-OL ',
 'FORMEL 1 . Verstappen tryner alle på anonym Magnussen-dag i Japan . Ingen kunne følge Red Bull-verdensmesteren på Suzu

In [1768]:

# new_train = []
# new_labels = []
# for t in range(len(train_data)):
#     if "De frafaldne: Hun vandt bronze i Rio" in train_data[t]:

#         print(train_data[t])
#     else:
#         new_train.append(train_data[t])
#         new_labels.append(train_labels[t])

# train_data = np.array(new_train)
# train_labels = np.array(new_labels)

# print(len(train_data))





In [1769]:

def combine_articles_to_csv():
    df_sport_combined = df_sport.copy().drop('Link', axis=1)
    df_sport_combined.to_csv('articles_temp/combined.csv')

combine_articles_to_csv()


In [1770]:
duplicate_rows = df_sport.duplicated()
print("Duplicates in data points: ")
print(df_sport[duplicate_rows])

Duplicates in data points: 
Empty DataFrame
Columns: [Category, Headline, SubHeading, Link, isResult, isMaybe]
Index: []


In [1771]:
import time
isin_dict = False
def test_lookup_performance():
    word_to_check = "Dansk"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)
    print(isin_dict)

test_lookup_performance()

# isin_dict


False


In [1772]:


# train_text = df_sport.iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()
# train_text_results = df_sport.loc[df_sport['isResult'] == True].iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()

from create_vocab import split_sentences
from create_vocab import remove_duplicates
from create_vocab import remove_nationalities
from create_vocab import remove_danske_navne
from create_vocab import remove_danske_fornavne
from create_vocab import remove_numeric
from create_vocab import remove_non_dict_words
from create_vocab import add_non_alpha_numeric

print(len(train_data))

words_arr = split_sentences(train_data)
words_arr_unique = remove_duplicates(words_arr)
words_arr_unique = remove_nationalities(words_arr_unique, nationalities)
words_arr_unique = remove_danske_navne(words_arr_unique, danske_navne)
words_arr_unique = remove_danske_fornavne(words_arr_unique, danske_fornavne)
words_arr_unique = remove_numeric(words_arr_unique)

words_train_vocab, words_sport_lingo = remove_non_dict_words(words_arr_unique, ordered_dict)



# # TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# # TODO : søg i alle leksikoner, søg med og uden bindestreg
# # TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)
# # TODO : lemmatizer : udelad bøjninger af samme navneord. eks : verdensmester/verdensmesteren
# # TODO : evt. grupper ord der ofte hænger sammen med nltk BigramFinder. eks vandt over
# TODO : fjern evt. også alle navne (fornavne og efternavne)  

print("total unique words:", len(words_arr_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total vocab:", len(words_train_vocab))
print("total articles:", len(df_sport) )

# for d in df_sport['isResult']:
#     if isinstance(d, bool) != True:
#         print(d)

1958


total unique words: 8378
total sports lingo words: 1843
total vocab: 6535
total articles: 2061


In [1773]:
file = open('words_sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()

file = open('words_train_vocab.txt','w')
for item in sorted(words_train_vocab):
	file.write(item+"\n")
file.close()


In [1774]:

def most_frequent_words_in_arr(arr):
    xx = np.array(arr, dtype=object)
    unique, counts = np.unique(xx, return_counts=True)
    aa = np.asarray((unique, counts)).T
    return np.flip(aa[aa[:, 1].argsort()])

frequent_words = most_frequent_words_in_arr(words_arr)

# TODO : lav en negativ liste også
# display most frequent words found in lingo words
for f in frequent_words:
    if f[1] in words_sport_lingo and f[1] not in navne and len(f[1]) > 1:
        print(f[1])


yankees
astros
sox
tyler
dodgers
nationals
hollywood
phillies
rodriguez
ruth
castro
isabella
skaggs
ortiz
field
guillermo
forbudte
giancarlo
beckers
beograd
wilson
biden
wade
bjerre
alberto
alyssa
dwyane
earvin
contador
chun
davide
vojens
cuba
rebellin
nehwal
texas
southlake
steinbrenners
stanton
saina
scherzer
robinson
royals
selig
kjer
kansas
happ
hudson
magic
mota
mets
letlands


In [1775]:


# print(frequent_words[:300])

In [1776]:


df_sport_labels = df_sport['isResult']

results_true = df_sport_labels.loc[df_sport_labels== True]
results_false = df_sport_labels.loc[df_sport_labels == False]

assert(len(results_true) + len(results_false) == len(df_sport_labels))

labels = df_sport_labels.to_numpy().astype(int)

print("Labels True: " , len(results_true))
print("Labels False: ", len(results_false))


Labels True:  980
Labels False:  1081


In [1777]:
# assert (len(labels) == len(train_text))
# print("Data:")
# for t in range(len(train_text)):
#     if labels[t] == 0:
#         print(train_text[t])
#         print(labels[t], "\n")

In [1778]:


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from vectorization import replace_finals, replace_nationality, to_lower
from vectorization import split_dash
from vectorization import split_included_specials
from vectorization import replace_tournament
from vectorization import replace_countries
from vectorization import replace_weekday
from vectorization import replace_digits
from vectorization import vect_layer_2_text
from vectorization import vectorize_layer
from vectorization import standardize
from static_data import tournaments
from static_data import weekdays
from static_data import non_alpha
from static_data import word_generalization


# TODO : evt indikere hvilke navneord der starte med stort bogstav(egenavne), evt. lave et opslag for at undersøge ordklasse for det første ord i sætningen 
# TODO : test hvilke standarization funktioner giver bedre resultater 

arrs = [
    to_lower, 
    split_dash, 
    split_included_specials, 
    replace_tournament(tournaments), 
    replace_countries(countries), 
    replace_weekday(weekdays), 
    replace_finals, 
    replace_nationality(nationalities),
    replace_digits
]

s = standardize(arrs)

words_train_vocab.extend(word_generalization)
words_train_vocab.extend(non_alpha)

# Model constants.
max_features = 6500
sequence_length = 100

vectorized_layer = vectorize_layer(max_features, sequence_length, s)

text_ds = vectorized_layer.adapt(words_train_vocab)
vect_vocab = vectorized_layer.get_vocabulary()

print("Total vocab/max_features : ",  len(vect_vocab))
# print (vect_layer_2_text(vectorized_layer(["OL Paris 2023 Fodbold superliga pokalen maraton-majors maraton-maj åmaj majå torsdags torsdage superligaer pokalfinaler semifinaler finalen finalerne"]), vect_vocab))

# print (vect_layer_2_text(vectorized_layer(["majoren bla major league bla major"]), vect_vocab))
# print (vect_layer_2_text(vectorized_layer(["pga PGA pga-turneringerne pga turneringens PGA-turneringen pga-turneringer turnering turneringer"]), vect_vocab))

print (vect_layer_2_text(vectorized_layer(["vuelta a españa bla 1933 _i .den 10."]), vect_vocab))

# is_match = tf.strings.regex_replace(
#     "1820. Tomorrow is 1938 another day 1938", pattern=r'(?:18|19|20)\d{2}', rewrite=r'x_year', replace_global=True
# )

# is_match

# TODO : Test for all non alpha numeric signs , eg », «

# print(weekdays)

print (vect_layer_2_text(vectorized_layer(["i juli månded Udesejr giver Team Esbjerg oprejsning efter sjælden nedtur "]), vect_vocab))



Total vocab/max_features :  6480
['xtournament' '[UNK]' 'xyear' 'i' '.' 'den' 'xnumber' '.']
['i' 'xweekday' '[UNK]' 'udesejr' 'giver' 'team' '[UNK]' 'oprejsning'
 'efter' 'sjælden' 'nedtur']


In [1779]:
for t in train_data[0:50]:
    print("Original \n:", t)
    print("Text from vectorized: \n", vect_layer_2_text(
        vectorized_layer([t]), vect_vocab
        ))
    print("\n")


Original 
: TENNIS . Efter stor forvirring sætter Holger Rune endelig navn på træner . Den danske tennisspiller Holger Rune skal igen trænes af sin barndomstræner Lars Christensen 
Text from vectorized: 
 ['tennis' '.' 'efter' 'stor' 'forvirring' 'sætter' '[UNK]' '[UNK]'
 'endelig' 'navn' 'på' 'træner' '.' 'den' 'xnationality' 'tennisspiller'
 '[UNK]' '[UNK]' 'skal' 'igen' 'trænes' 'af' 'sin' 'barndomstræner'
 '[UNK]' '[UNK]']


Original 
: EM HÅNDBOLD . EM-nederlag nager stadig verdensmestrene: 'Gør ekstra ondt mod Danmark' . Katrine Lunde og Henny Reistad er endnu ikke kommet sig helt over EM-nederlaget til Danmark 
Text from vectorized: 
 ['xtournament' 'håndbold' '.' 'xtournament' 'nederlag' 'nager' 'stadig'
 'verdensmestrene' ':' "'" 'gør' 'ekstra' 'ondt' 'mod' 'xland' "'" '.'
 '[UNK]' 'lunde' 'og' '[UNK]' '[UNK]' 'er' 'endnu' 'ikke' 'kommet' 'sig'
 'helt' 'over' 'xtournament' 'nederlaget' 'til' 'xland']


Original 
: HERRELANDSHOLDET . Landsholdet elsker kritikken af dårligt spil

In [1780]:
# def split_data(data, labels, percentage):
#     l = len(data)
#     p = l - int((percentage/100) * l)
#     return (data[0:p], data[p:], labels[0:p], labels[p:])


# train_data, val_data, train_labels, val_labels = split_data(vectorize_layer(train_text), labels, 7)

# print("Total data: ", len(train_text))
# print("Train data length: ", len(train_data))
# print("Validation data length: ", len(val_data))


In [1781]:
train_data_vect = vectorized_layer(train_data)
val_data_vect = vectorized_layer(val_data)



In [1782]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [1783]:
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [1784]:
from tensorflow.keras import layers
import random as python_random


def get_transformer_model():

    embed_dim =  192 # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 256  # Hidden layer size in feed forward network inside transformer


    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    # x = layers.Embedding(max_features, embed_dim)(inputs)

    embedding_layer = TokenAndPositionEmbedding(sequence_length, max_features, embed_dim)
    x = embedding_layer(inputs)
    # x = layers.Dropout(0.2)(x)

    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    # x = layers.Dropout(0.1)(x)

    # x = transformer_block(x)



    # Conv1D + global max pooling
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(32, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)


    transformer_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    transformer_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return transformer_model


In [1785]:
from tensorflow.keras import layers
import random as python_random

def get_cnn_model():

    embedding_dim = 192

    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(None,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    x = layers.Dropout(0.2)(x)

    # Conv1D + global max pooling
    x = layers.Conv1D(256, 6, padding="valid", activation="relu", strides=2)(x)
    # x = layers.Conv1D(256, 6, padding="valid", activation="relu", strides=2)(x)
    # x = layers.Conv1D(256, 6, padding="valid", activation="relu", strides=2)(x)

    x = layers.Conv1D(96, 4, padding="valid", activation="relu", strides=2)(x)

    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
    
    cnn_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return cnn_model



In [1786]:

def prepare_model(name):
    if (name == "cnn"):
       return get_cnn_model()
    elif (name == "transformer"):
       return get_transformer_model()
  

def filter_max_accuracy(history, threshold = 0.95):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    list = []
    for x in range(len(acc)):
        if (acc[x] > threshold):
            list.append(val_acc[x])

    return np.array(list)

models = ["cnn", "transformer"]

callback_3_loss = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)


def mean_model_accuracy(mode_names, iterations, epochs = 20):

  
    results = []

    for name in range(len(mode_names)):
        model_name = mode_names[name]
        val_accuracies = []
        
        for x in range(iterations):
            model = prepare_model(model_name)

            # Fit the model using the train and test datasets.
            history = model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

            max_val_acc = filter_max_accuracy(history)
            val_accuracies.append(max(max_val_acc))
            print(max(max_val_acc))
            print(val_accuracies)
        
        d = dict(name = model_name, results = np.mean(np.squeeze(np.array(val_accuracies))))
        results.append(d)
        
    return results


In [1787]:
# mean_results = mean_model_accuracy(models, 8)
# mean_results

In [1788]:
def result_format_round(result):
    return round(result)

def result_format_none(result):
    return result

def print_model_score(model):
    score = model.evaluate(val_data_vect, val_labels, verbose=0)
    print("Validation loss:", score[0])
    print("Validations accuracy:", score[1])

def print_validation_results(predictions, val_data, labels, formatter, only_incorrect = True):
    print("Number of predictions", len(predictions))
    n_correct = 0
    for x in range(len(val_data)):
        correct_prediction = result_format_round(labels[x]) == result_format_round(predictions[x][0])
        if correct_prediction:
            n_correct += 1

        if correct_prediction == False:
            print("VALIDATION SAMPLE TEXT: \n" ,val_data[x])
            print("VALIDATION SAMPLE DE-VECTORIZED: \n" ,vect_layer_2_text(val_data_vect[x], vect_vocab))
            print("LABEL --:" , formatter(labels[x]), " ---- float: ", labels[x])
            print("PREDICTION --:" , formatter(predictions[x][0]), " ---- float: ", predictions[x][0])
            print("CORRECT PREDICTION: ", correct_prediction)
            print("\n")
    
    print("Number correct: ", n_correct)

In [1789]:

epochs= 4
transformer_model = get_transformer_model()

# Fit the model using the train and test datasets.
transformer_history = transformer_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=10, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

Epoch 1/4
196/196 [==============================] - 4s 15ms/step - loss: 0.5485 - accuracy: 0.7206 - val_loss: 0.3373 - val_accuracy: 0.8155
Epoch 2/4
196/196 [==============================] - 3s 14ms/step - loss: 0.1517 - accuracy: 0.9525 - val_loss: 0.3057 - val_accuracy: 0.8932
Epoch 3/4
196/196 [==============================] - 3s 14ms/step - loss: 0.0178 - accuracy: 0.9980 - val_loss: 0.3797 - val_accuracy: 0.8641
Epoch 4/4
196/196 [==============================] - 3s 14ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.3667 - val_accuracy: 0.8738


In [1790]:

# epochs= 12
# cnn_model = get_cnn_model()


# # # Fit the model using the train and test datasets.
# transformer_history = cnn_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=12, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

In [1791]:
def print_results(model):
    np.set_printoptions(precision = 5, suppress = True)
    predictions = model.predict(val_data_vect)
    print_model_score(model)
    print("\n")
    print_validation_results(predictions, val_data, val_labels, result_format_round)
  

In [1792]:
print("--- TRANSFORMER ---")
print_results(transformer_model)

--- TRANSFORMER ---
Validation loss: 0.36674830317497253
Validations accuracy: 0.8737863898277283


Number of predictions 103
VALIDATION SAMPLE TEXT: 
 SPORT . Thorbjørn erobrer USA med tro på sejr . Thorbjørn Olesen jagter målrettet en sejr i 2014 på den amerikanske PGA-tour 
VALIDATION SAMPLE DE-VECTORIZED: 
 ['sport' '.' '[UNK]' 'erobrer' 'xland' 'med' 'tro' 'på' 'sejr' '.' '[UNK]'
 '[UNK]' 'jagter' 'målrettet' 'en' 'sejr' 'i' 'xyear' 'på' 'den'
 'xnationality' 'xtournament']
LABEL --: 0  ---- float:  0
PREDICTION --: 1  ---- float:  0.97784525
CORRECT PREDICTION:  False


VALIDATION SAMPLE TEXT: 
 HERRELANDSHOLDET . Kraul godkender plænen i Dublin: Den er som et billardklæde . Banen i Parken var en katastrofe, lød det fra landsholdet efter 0-0 mod Irland. DR Sportens udsendte, Andreas Kraul, inspicerer plænen i Dublin før VM-returgyseren 
VALIDATION SAMPLE DE-VECTORIZED: 
 ['herrelandsholdet' '.' '[UNK]' 'godkender' 'plænen' 'i' '[UNK]' ':' 'den'
 'er' 'som' 'et' '[UNK]' '.' 'banen

In [1793]:

# print("--- CNN ---")
# print_results(cnn_model)

In [1794]:
import os

# Set up a logs directory, so Tensorboard knows where to look for files.

ll = transformer_model.layers[1]
ll_weights = ll.get_weights()[0]

# print(ll_weights.shape)
ll_weights


array([[-0.02235, -0.00371,  0.00637, ..., -0.04167,  0.03918,  0.01946],
       [-0.03298,  0.04034,  0.0367 , ..., -0.04357,  0.05792,  0.02217],
       [-0.01325, -0.03435, -0.01805, ..., -0.00064,  0.01981, -0.07423],
       ...,
       [-0.00192,  0.00694, -0.03578, ..., -0.04114, -0.00904,  0.02336],
       [ 0.01015, -0.00275, -0.04063, ...,  0.02653, -0.01543, -0.04849],
       [ 0.00741, -0.02517,  0.01089, ...,  0.04958,  0.01123, -0.01582]],
      dtype=float32)

In [1795]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, len(vect_vocab)):
  word = vect_vocab[index]  # flipping the key-value in word_index
  embeddings = ll_weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [1796]:
# from nltk import collocations
# bigram_measures = collocations.BigramAssocMeasures()
# finder = collocations.BigramCollocationFinder.from_words(["New", "York", "is", "big", "New", "York", "is", "dirty"])
# finder.ngram_fd.items()



In [1797]:
# import lemmy
# # Create an instance of the standalone lemmatizer.
# lemmatizer = lemmy.load("da")

# # Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
# lemmatizer.lemmatize("NOUN", "verdensetter")



In [1798]:
# import nltk as nltk
# # from string import punctuation
# # from nltk.corpus import stopwords
# # nltk.download('stopwords')

# # da_stopwords = stopwords.words("danish")


In [1799]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorized_layer(inputs)
# Turn vocab indices into predictions
outputs = transformer_model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="transformer_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [1800]:
print("\nResults:")

s = "SPORT Hun vandt bronze i mandags . Roer Anne Dsane Andersen har som 24-årig vundet bronze ved OL"

print(end_to_end_model.predict(
    [
      "Fodbold Fjerritslev vinder over Vordingborg. Træner kommenterer på historisk kamp",
       s
     ]))

print(vect_layer_2_text(vectorized_layer([s]), vect_vocab))
print(vect_layer_2_text(vectorized_layer(["Fodbold Flere forventer at Fjerritslev vinder over Vordingborg. 'Fjerritslev vinder!', udtaler hjemmeholdets træner inden historisk kamp"]), vect_vocab))


print("\n NON-Results:") 
print(end_to_end_model.predict(
    [
      "Fodbold Træner for Fjerritslev ser frem til sejr over Vordingborg. 'Det bliver en historisk kamp'",
      "Fodbold De danske spillere skal op imod Sverige, som de tabte til i 2022",
      "Fodbold De danske spillere forsøge at besejre Tyrkiet den kommende Lørdag i VM-kamp. Tyrkiet har aldriv været i en VM-finale",
      "Fodbold De danske spillere tror på sejr mod Tyrkiet. 'Den skal vindes'",
      "Skisport Sverige drømmer om flere medaljer og sejre til næste års OL. Træner forventer flere rekorder",
      "Skisport Buller vil overraske alle og gøre det umulige. 'Jeg vinder i VM'",
      "Badminton . Axelsen reagerer på nederlag . Viktor Axelsen tog et par dage fri efter nedturen",
     ]))


Results:
[[0.8945 ]
 [0.99393]]
['sport' 'hun' 'vandt' 'bronze' 'i' 'xweekday' '.' 'roer' '[UNK]' '[UNK]'
 '[UNK]' 'har' 'som' 'xnumber' 'årig' 'vundet' 'bronze' 'ved'
 'xtournament']
['fodbold' 'flere' 'forventer' 'at' '[UNK]' 'vinder' 'over' '[UNK]' '.'
 "'" '[UNK]' 'vinder' '!' "'" 'udtaler' 'hjemmeholdets' 'træner' 'inden'
 'historisk' 'kamp']

 NON-Results:
[[0.06319]
 [0.09566]
 [0.00785]
 [0.01659]
 [0.02928]
 [0.5945 ]
 [0.99963]]
